In [1]:
import datetime
import warnings
# warnings.filterwarnings('ignore')

import pandas as pd
from binance.spot import Spot
from binance.websocket.spot.websocket_stream import SpotWebsocketStreamClient
# WebSocket API Client
from binance.websocket.spot.websocket_api import SpotWebsocketAPIClient
from Keys import *
from MultiTrade.live_environments import LiveTradingEnv
from neuralforecast.core import NeuralForecast
from ray.rllib.policy.policy import Policy
from IPython. display import display
from pprint import pprint
from utils.utils import build_market_image, sharpe_reward, symbol_map, binanace_col_map
import json
import ast
import sqlite3 as db
from utils.rendering import LiveRenderer
import subprocess
import time
import threading

In [2]:
image=build_market_image(target_pair='ETH/USDT',time_frame='1h',axis=0)

In [3]:
datetime. datetime.now()-pd.Timedelta('1h').to_pytimedelta()

datetime.datetime(2024, 5, 29, 17, 5, 43, 718258)

In [4]:
trade_target='BTC/USDT'
api_key=binanace_api_key
api_secret=binance_api_secret

client =Spot(api_key=api_key,api_secret=api_secret,base_url='https://testnet.binance.vision')
client


In [5]:
binanace_col_map={
                    "t": "date_open",
                    "T": "date_close",
                    "s": "symbol",
                    "o": "open",
                    "c": "close",
                    "h": "high",
                    "l": "low",
                    "v": "volume",
                    "x": "is_closed",
                }

In [6]:
import pytorch_lightning
import logging


In [7]:
pytorch_lightning._logger.setLevel(0)

In [8]:
import logging
logging.getLogger('lightning').setLevel(0)

In [9]:
symbol="ETHUSDT"
history_path='Trade_history/trade.db'

conn=db.connect(history_path)

In [10]:

model=NeuralForecast.load("forecasting_model",verbose=False)


Seed set to 5
Seed set to 11
Seed set to 17
Seed set to 6


In [11]:
agent_dir='Agent/final_checkpoints/policies/default_policy'
agent= Policy.from_checkpoint(agent_dir)

In [12]:
live_env_config=dict(
                name='ETHUSDT_train',
                model=model,
                agent=agent,
                api_key=api_key,
                api_secret=api_secret,
                test_net=True,
                restore_trading=True,
                target_symbol='ETH/USDT',
                time_frame='1h',
                reward_function=sharpe_reward,
                positions = [ -.5,-.25,.25, .5], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
                trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
                borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
                max_episode_duration=168,
                verbose=0,
                )

In [13]:
self=LiveTradingEnv(**live_env_config)
self.df
obs,info=self.reset(reset_account=False)

In [14]:

def message_handler(client, message):
    try:
        print(message)
        message_data=json.loads(message)
        k_data=message_data['k']


        # print(type(data),data)
        data=pd.DataFrame([k_data])
        
        data.columns=data.columns.map(binanace_col_map)
        data=data[[c for c in binanace_col_map.values() if c in data.columns]]
        display(data)
        data=data.drop('date_open',axis=1)
        data["date_close"]=pd.to_datetime(data["date_close"],unit='ms')
        data=data.set_index('date_close')
        data['symbol'] = self.symbol
        data['unique_id']=symbol_map[self.symbol]

        
        conn=db.connect(history_path)
        data.to_sql(f'{self.symbol}_candle_history',conn,if_exists='append',index=False)
        

        if k_data['x']:
            data=self.get_data()
            self._set_df(data)
            self._prep_forecasts()
            self._set_df(data)

            obs = self._get_obs()
            
            action,_,states=self.agent.compute_single_action(obs,explore=False)
            obs, reward, terminated, truncated, info=self.live_step(action,wait=False)

            print(obs.shape)

    except Exception as e: 
        print('bad_data',message)
        print(e)

In [15]:
def on_start(message):
        obs,info=self.reset(reset_account=False)
        start_time=datetime. datetime.now().strftime("%m-%d-%Y %H:%M")
        print (f'START trading session at {start_time}')
        action,_,states=self.agent.compute_single_action(obs,explore=False)
        obs, reward, terminated, truncated, info=self.live_step(action,wait=False)

def on_close(_):
        end_time=datetime. datetime.now().strftime("%m-%d-%Y %H:%M")
        print (f'FINISH trading session at {end_time}')
        self.reset_account()
        

In [16]:
# my_client = SpotWebsocketAPIClient(api_key=api_key,api_secret=api_secret,on_message=self.stream_step,stream_url='wss://testnet.binance.vision')
my_client = SpotWebsocketStreamClient(
                                    on_close=on_close,
                                    on_open=on_start,
                                    on_error=on_close,
                                    on_message=self.stream_step,
                                    stream_url='wss://testnet.binance.vision')
renderer=LiveRenderer('Trade_history/trade.db')

START trading session at 05-29-2024 18:05
stay in position


In [17]:
import threading
# Subscribe to a single symbol stream
app=threading.Thread(target=renderer.run)

for i in range(7):
    # my_client = SpotWebsocketAPIClient(api_key=api_key,api_secret=api_secret,on_message=self.stream_step,stream_url='wss://testnet.binance.vision')
    my_client = SpotWebsocketStreamClient(
                                        on_close=on_close,
                                        on_open=on_start,
                                        on_error=on_close,
                                        on_message=self.stream_step,
                                        stream_url='wss://testnet.binance.vision')
    wait_time=pd.Timedelta(hours=24).total_seconds()
    my_client.kline(symbol="ETHUSDT",interval="1h")
    # app=threading.Thread(target=renderer.run,kwargs=dict(symbol="ETHUSDT",interval="1h"))
    app.start()
    print('waiting to reconnect')
    time.sleep(wait_time)



START trading session at 05-29-2024 18:05
stay in position
waiting to reconnect
 * Serving Flask app 'gym_trading_env.renderer'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
Exception in thread Thread-13 (run):
Traceback (most recent call last):
  File "/Users/sebastiancoombs/anaconda3/envs/agent_env/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/sebastiancoombs/anaconda3/envs/agent_env/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "/Users/sebastiancoombs/anaconda3/envs/agent_env/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/sebastiancoombs/Documents/Git/MultiTrader/utils/rendering.py", line 69, in run
    self.app.run()
  File "/Users/sebastiancoombs/anaconda3/envs/agent_env/lib/python3.10/site-packages/flask/app.py", line 625, in run
    run_simple(t.cast(str, host), port, self, **options)
  File "/Users/sebastiancoombs/anaconda3/envs/agent_env/lib/python3.10/site-packages/werkzeug/serving.py", line 1106, in run_simple
    r

bad_data {"result":null,"id":1717031156399}
'k'


In [ ]:
pd.Timestamp(1717013599913,unit='ms')

In [ ]:
# client.avg_price('ETHUSDT'),client.ticker_price('ETHUSDT')


In [ ]:
obs,info=self.reset(reset_account=False)
obs.shape


In [ ]:
from ray.rllib.policy.policy import Policy
from ray.rllib.algorithms.algorithm import Algorithm


In [ ]:
agent_dir='Agent/final_checkpoints/policies/default_policy'
agent= Policy.from_checkpoint(agent_dir)

# agent.export_model('Agent/final_checkpoints/pretrained_agent')


In [ ]:
# action,_,states=agent.compute_single_action(obs)
# obs, reward, terminated, truncated, info=self.live_step(action,wait=False)
# action,_,states=agent.compute_single_action(obs)


In [ ]:
obs,info=self.reset(reset_account=True)
i=0
for i in range(10):
    action,_,states=agent.compute_single_action(obs,explore=False)
    obs, reward, terminated, truncated, info=self.live_step(action,wait=False)

In [ ]:
# h=self.historical_info[0]
history_df=pd.DataFrame([self.historical_info[-1]])
history_df

In [ ]:
import sqlite3 as db
import pandas as pd



# connect to a test DB with one three-column table titled "Demo"
conn = db.connect('Trade_history/trade.db')
cur = conn .cursor()

In [ ]:
history_df=pd.read_sql(f'select * from {symbol}_candle_history',conn)
history_df

In [ ]:
history_df.to_sql(f'{self.symbol}_candle_history',conn,if_exists='append')

In [ ]:
self.historical_info[-1]

In [ ]:
for h in self.historical_info:
    insert_sql = f'INSERT INTO {self.symbol}_trade_history {h}' 
    conn.execute(insert_sql)

In [ ]:
client.klines("BNBUSDT", "1h", limit=1)

In [ ]:
self._get_ticker()

In [ ]:
data_list=client.klines("BNBUSDT", "1h", limit=10)
columns=['date_open','open','high','low','close','volume','date_close','QA_volume','N_trades','BA_volume','BQ_volume','unused']
data=pd.DataFrame(data_list,columns=columns)
data['date_close']=data['date_close'].apply(pd.to_datetime,unit='ms')
data['date_open']=data['date_open'].apply(pd.to_datetime,unit='ms')

# data_list

In [ ]:
data